In [2]:
import boto3
import os

In [ ]:
os.environ.setdefault("AWS_PROFILE", "aws-workshop-dev-user")

In [ ]:
os.environ.setdefault("AWS_PROFILE", "aws-workshop-prd-user")

In [6]:
s3_client = boto3.client('s3')

In [23]:
objects = s3_client.list_objects(Bucket="aws-workshop-dev-rmc-2023")

In [27]:
objects["Contents"]

for file in objects["Contents"]:
    print(file["Key"])

tf
tf/config.tf
tf/main.tf
tf/terraform.tfstate
